In [ ]:
# 운영 DB 적재에 해당된다

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
# 데이터마트용 계정 생성
# mysql> create user bigSVC@'%' identified by 'bigSVC1234@';
# Query OK, 0 rows affected (0.01 sec)

# mysql> grant all privileges on *.* to bigSVC@'%' with grant option;
# - with grant option : 해당 계정이 받은 권한을 다른 계쩡에게 전달해주는게 가능하게 하겠다
# Query OK, 0 rows affected (0.02 sec)


In [4]:
from datetime import date, datetime
from pyspark.sql.types import *
from pyspark.sql.functions import *
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt #그래프 패키지 모듈 등록
%matplotlib inline 


In [5]:
def cal_std_day(befor_day):   
    x = dt.datetime.now() - dt.timedelta(befor_day)
    year = x.year
    month = x.month if x.month >= 10 else '0'+ str(x.month)
    day = x.day if x.day >= 10 else '0'+ str(x.day)  
    return str(year)+ '-' +str(month)+ '-' +str(day)

### JDBC 연결정보

In [59]:
# DM DB 연결정보
conf_dm = {
      'url':'jdbc:mysql://localhost:3306/etlMysqlDM?characterEncoding=utf8&serverTimezone=Asia/Seoul'
     ,'props':{
      'user':'bigMysql',
      'password':'bigMysql1234@'   
      }
}

# service DB 연결정보
# conf_svc= {
#       'url':'jdbc:mysql://localhost:3306/etlMysqlSVC?characterEncoding=utf8&serverTimezone=Asia/Seoul'
#      ,'props':{
#       'user':'bigSVC',
#       'password':'bigSVCl1234@'   
#       }
# }

conf_svc = {
    'url': 'jdbc:mysql://127.0.0.1:3306/etlMysqlSVC?characterEncoding=utf8&serverTimezone=Asia/Seoul',
    'props': {
        'user': 'bigSVC',
        'password': 'bigSVCl1234@'
    }
}


## Load DataMart

### 1. 지역별 단위면적(km)당 인구수와 코로나 확진자 수 데이터 저장
- base data load
    - LOC table : 지역별 인구와 면적
    - dm db data read -> 처리코드 -> service db dataload

In [60]:
spark.read.jdbc(url=conf_dm["url"], table = "LOC", properties=conf_dm["props"]).show()

+----+------+---------+
| LOC|  AREA|     POPU|
+----+------+---------+
|서울|  605 | 9736027 |
|부산|  770 | 3396109 |
|대구|  883 | 2412642 |
|인천| 1065 | 3014739 |
|광주|  501 | 1462545 |
|대전|  540 | 1469543 |
|울산| 1062 | 1138419 |
|세종|  465 |  376779 |
|경기|10195 |13925862 |
|강원|16830 | 1555876 |
|충북| 7407 | 1633472 |
|충남| 8246 | 2181835 |
|전북| 8070 | 1817186 |
|전남|12348 | 1865459 |
|경북|19034 | 2677709 |
|경남|10541 | 3377331 |
|제주| 1850 |  697476 |
+----+------+---------+



In [61]:
# db table 읽기, 저장함수
# read 함수
def find_data(config, table_name) :
    return spark.read.jdbc(url=config["url"], table = table_name, properties=config["props"])

# write 함수
def save_data(config, df, table_name) :
    df.write.jdbc(url=config["url"], table = table_name, mode="append", properties=config["props"])

#### 단위 면적당 인구수, 코로나 확진자 수 현황
- DM DB의 LOC, CORPNA_PATIENTS 테이블

In [62]:
popu = find_data(conf_dm, "LOC")
patients = find_data(conf_dm, "CORONA_PATIENTS")

In [63]:
popu.columns
patients.columns

['LOC', 'AREA', 'POPU']

['LOC', 'DEATH_CNT', 'DEF_CNT', 'LOC_OCC_CNT', 'QUR_RATE', 'STD_DAY']

['LOC', 'DEATH_CNT', 'DEF_CNT', 'LOC_OCC_CNT', 'QUR_RATE', 'STD_DAY']   
 지역   누적사망자수  누적 발생수  지역발생수     만명당 발생률  기준일자

In [64]:
popu.count()
patients.count()

17

204

In [65]:
popu.join(patients, on="LOC").tail(10)
popPatient = popu.join(patients, on="LOC") \
                 .select("LOC",
                        ceil(col("POPU")/col("AREA")).alias("POPU_DENST"),
                        "QUR_RATE", "STD_DAY") \
                 .orderBy(col("STD_DAY"))

[Row(LOC='충북', AREA='7407 ', POPU='1633472 ', DEATH_CNT='123', DEF_CNT='12555', LOC_OCC_CNT='55', QUR_RATE='786', STD_DAY='2022-01-18'),
 Row(LOC='충북', AREA='7407 ', POPU='1633472 ', DEATH_CNT='123', DEF_CNT='12636', LOC_OCC_CNT='79', QUR_RATE='791', STD_DAY='2022-01-19'),
 Row(LOC='충북', AREA='7407 ', POPU='1633472 ', DEATH_CNT='123', DEF_CNT='12812', LOC_OCC_CNT='174', QUR_RATE='802', STD_DAY='2022-01-20'),
 Row(LOC='충북', AREA='7407 ', POPU='1633472 ', DEATH_CNT='123', DEF_CNT='12898', LOC_OCC_CNT='85', QUR_RATE='807', STD_DAY='2022-01-21'),
 Row(LOC='충북', AREA='7407 ', POPU='1633472 ', DEATH_CNT='123', DEF_CNT='12636', LOC_OCC_CNT='79', QUR_RATE='791', STD_DAY='2022-01-19'),
 Row(LOC='충북', AREA='7407 ', POPU='1633472 ', DEATH_CNT='123', DEF_CNT='12812', LOC_OCC_CNT='174', QUR_RATE='802', STD_DAY='2022-01-20'),
 Row(LOC='충북', AREA='7407 ', POPU='1633472 ', DEATH_CNT='123', DEF_CNT='12898', LOC_OCC_CNT='85', QUR_RATE='807', STD_DAY='2022-01-21'),
 Row(LOC='충북', AREA='7407 ', POPU='1633

In [66]:
popPatient.show(5)
popPatient.tail(5)

+----+----------+--------+----------+
| LOC|POPU_DENST|QUR_RATE|   STD_DAY|
+----+----------+--------+----------+
|경남|       321|     679|2022-01-16|
|대구|      2733|    1040|2022-01-16|
|경북|       141|     637|2022-01-16|
|경남|       321|     679|2022-01-16|
|경북|       141|     637|2022-01-16|
+----+----------+--------+----------+
only showing top 5 rows



[Row(LOC='제주', POPU_DENST=378, QUR_RATE='730', STD_DAY='2022-01-21'),
 Row(LOC='충남', POPU_DENST=265, QUR_RATE='978', STD_DAY='2022-01-21'),
 Row(LOC='충남', POPU_DENST=265, QUR_RATE='978', STD_DAY='2022-01-21'),
 Row(LOC='충북', POPU_DENST=221, QUR_RATE='807', STD_DAY='2022-01-21'),
 Row(LOC='충북', POPU_DENST=221, QUR_RATE='807', STD_DAY='2022-01-21')]

- 각 DB에 고유번호를 추가해서 레코드를 유일하게 구별하는 컬럼으로 사용
    - spark 데이터 프레임의 statfuctions 패키지 함수 중 monotonically_increasing_id
    - 데이터프레임의 로우에 매핑되는 고유 id 반환

In [67]:
popPatFin = popPatient.withColumn("CP_IDX", monotonically_increasing_id())
popPatFin.show()

+----+----------+--------+----------+------+
| LOC|POPU_DENST|QUR_RATE|   STD_DAY|CP_IDX|
+----+----------+--------+----------+------+
|강원|        93|     888|2022-01-16|     0|
|강원|        93|     888|2022-01-16|     1|
|경기|      1366|    1534|2022-01-16|     2|
|경기|      1366|    1534|2022-01-16|     3|
|경남|       321|     679|2022-01-16|     4|
|경남|       321|     679|2022-01-16|     5|
|경북|       141|     637|2022-01-16|     6|
|경북|       141|     637|2022-01-16|     7|
|광주|      2920|     697|2022-01-16|     8|
|광주|      2920|     697|2022-01-16|     9|
|대구|      2733|    1040|2022-01-16|    10|
|대구|      2733|    1040|2022-01-16|    11|
|대전|      2722|     900|2022-01-16|    12|
|대전|      2722|     900|2022-01-16|    13|
|부산|      4411|     832|2022-01-16|    14|
|부산|      4411|     832|2022-01-16|    15|
|서울|     16093|    2543|2022-01-16|    16|
|서울|     16093|    2543|2022-01-16|    17|
|세종|       811|     634|2022-01-16|    18|
|세종|       811|     634|2022-01-16|    19|
+----

In [68]:
save_data(conf_svc, popPatFin, "CO_POP_DENSITY")

Py4JJavaError: An error occurred while calling o392.jdbc.
: java.sql.SQLException: Access denied for user 'bigSVC'@'localhost' (using password: YES)
	at com.mysql.cj.jdbc.exceptions.SQLError.createSQLException(SQLError.java:129)
	at com.mysql.cj.jdbc.exceptions.SQLExceptionsMapping.translateException(SQLExceptionsMapping.java:122)
	at com.mysql.cj.jdbc.ConnectionImpl.createNewIO(ConnectionImpl.java:828)
	at com.mysql.cj.jdbc.ConnectionImpl.<init>(ConnectionImpl.java:448)
	at com.mysql.cj.jdbc.ConnectionImpl.getInstance(ConnectionImpl.java:241)
	at com.mysql.cj.jdbc.NonRegisteringDriver.connect(NonRegisteringDriver.java:198)
	at org.apache.spark.sql.execution.datasources.jdbc.connection.BasicConnectionProvider.getConnection(BasicConnectionProvider.scala:49)
	at org.apache.spark.sql.execution.datasources.jdbc.connection.ConnectionProviderBase.create(ConnectionProvider.scala:102)
	at org.apache.spark.sql.jdbc.JdbcDialect.$anonfun$createConnectionFactory$1(JdbcDialects.scala:160)
	at org.a

### 2. 백신접종 완료자와 코로나 발생현황 데이터 저장
- 백신 접종 3차 접종자수(누적) : 인구 10만명당 접종자수
- 코로나 발생현황 : QUR_RATE(인구 10만명당 발생자수)

In [30]:
vaccine = find_data(conf_dm, "CORONA_VACCINE")
vaccine.show(3)
vaccine.printSchema()

+----+----------+-------+---------+
| loc|   std_day|level_2|    V_CNT|
+----+----------+-------+---------+
|대전|2022-01-16|     v2|1201356.0|
|대전|2022-01-16|     v1|  1230936|
|대전|2022-01-16|     v3|   8242.0|
+----+----------+-------+---------+
only showing top 3 rows

root
 |-- loc: string (nullable = true)
 |-- std_day: string (nullable = true)
 |-- level_2: string (nullable = true)
 |-- V_CNT: string (nullable = true)



In [31]:
vaccine.withColumn("V_CNT", vaccine["V_CNT"].cast(IntegerType()))
vaccine.show(2)

DataFrame[loc: string, std_day: string, level_2: string, V_CNT: int]

+----+----------+-------+---------+
| loc|   std_day|level_2|    V_CNT|
+----+----------+-------+---------+
|대전|2022-01-16|     v2|1201356.0|
|대전|2022-01-16|     v1|  1230936|
+----+----------+-------+---------+
only showing top 2 rows



In [48]:
# long df --> wide df 변환 (V_TH 컬럼 값을 컬럼으로 변환)
# pivot 사용 : pandas df로 변환
pdVac = vaccine.pandas_api()
type(pdVac)

pyspark.pandas.frame.DataFrame

In [49]:
pdVac

,loc,std_day,level_2,V_CNT
0,대전,2022-01-16,v2,1201356.0
1,대전,2022-01-16,v1,1230936
2,대전,2022-01-16,v3,8242.0
3,울산,2022-01-17,v2,930969.0
4,울산,2022-01-17,v1,956456
5,울산,2022-01-17,v3,240.0
6,서울,2022-01-16,v2,8078465.0
7,서울,2022-01-16,v1,8235942
8,서울,2022-01-16,v3,53011.0
9,전남,2022-01-13,v2,1593531.0


In [36]:
pdVacV1 = pdVac.pivot_table(index=["LOC", "STD_DAY"], columns="V_TH", values = "V_CNT")
pdVacV1.head()
pdVacV1 = pdVacV1.reset_index()
pdVacV1.head()

ValueError: Wrong columns V_TH.

- Spark 데이터 프레임의 StatFunctions 패키지 함수 중 monotonically_increasing_id
    - 
데이터 프레임의 로우에 되는매핑되는  고유 id 반환



### 2.  백신접종 완료자와 코로나 확진자 수 데이터 저장

In [39]:
vaccine = pdVacV1.to_spark()

NameError: name 'pdVacV1' is not defined

In [42]:
# 지역별 인구 10만명당 백신 접종률 계산
# 인구 data 필요
popu = find_data(conf_dm, 'LOC')
# 코로나 현황 data 필요
patients = find_data(conf_dm,'CORONA_PATIENTS')

In [43]:
vacRate = vaccine.join(popu, on="LOC") \
                 .select("LOC",
                         "STD_DAY",
                         ceil(col("v3")/col("POPU")*100000).alias("THRD_RATE"))
vacRate.show(1)

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `v3` cannot be resolved. Did you mean one of the following? [`loc`, `AREA`, `POPU`, `V_CNT`, `level_2`].;
'Project [LOC#185, STD_DAY#186, 'ceil((('v3 / POPU#299) * 100000)) AS THRD_RATE#321]
+- Project [loc#185, std_day#186, level_2#187, V_CNT#188, AREA#298, POPU#299]
   +- Join Inner, (loc#185 = LOC#297)
      :- Relation [loc#185,std_day#186,level_2#187,V_CNT#188] JDBCRelation(CORONA_VACCINE) [numPartitions=1]
      +- Relation [LOC#297,AREA#298,POPU#299] JDBCRelation(LOC) [numPartitions=1]


In [44]:
coPatVac = vacRate.join(patients, on = ["LOC", "STD_DAY"]) \
                  .select("LOC", "STD_DAY", "THRD_RATE", "QUR_RATE")

NameError: name 'vacRate' is not defined

In [45]:
coPatVac.show(3)

NameError: name 'coPatVac' is not defined

In [ ]:
coPatVac = coPatVac.withColumn("CV_IDX", monotonically_increasing_id())
coPatVac.show(2)

In [46]:
save_data(conf_svc, coPatVac, "CO_VACCINE_PATIENT")

NameError: name 'coPatVac' is not defined

### 3. 다중이용시설과 코로나 확진자 수의 상관관계
- DM DB LOC_FACILITY_CNT
    - 인구 10만명당 다중 이용시설 수
- DM DB CORONA_PATIENTS

In [50]:
facil = find_data(conf_dm, "LOC_FACILITY_CNT")
facil.show(1)

+----+-------+
| LOC|FAC_CNT|
+----+-------+
|경북|    944|
+----+-------+
only showing top 1 row



In [53]:
facPop = popu.join(facil, on = "LOC") \
             .select("LOC", 
                     ceil(facil.FAC_CNT/popu.POPU*100000).alias("FAC_POPU"))
facPop.show(1)

+----+--------+
| LOC|FAC_POPU|
+----+--------+
|강원|      38|
+----+--------+
only showing top 1 row



In [57]:
coFacPat = patients.join(facPop, on="LOC") \
                   .select("LOC", "FAC_POPU", "QUR_RATE", "STD_DAY") \
                   .withColumn("CF_IDX", monotonically_increasing_id())
coFacPat.show()

+----+--------+--------+----------+------+
| LOC|FAC_POPU|QUR_RATE|   STD_DAY|CF_IDX|
+----+--------+--------+----------+------+
|경북|      36|     641|2022-01-17|     0|
|경북|      36|     646|2022-01-18|     1|
|경북|      36|     637|2022-01-16|     2|
|경북|      36|     652|2022-01-19|     3|
|경북|      36|     661|2022-01-20|     4|
|경북|      36|     670|2022-01-21|     5|
|경북|      36|     652|2022-01-19|     6|
|경북|      36|     661|2022-01-20|     7|
|경북|      36|     670|2022-01-21|     8|
|경북|      36|     641|2022-01-17|     9|
|경북|      36|     646|2022-01-18|    10|
|경북|      36|     637|2022-01-16|    11|
|대전|      53|     913|2022-01-18|    12|
|대전|      53|     900|2022-01-16|    13|
|대전|      53|     906|2022-01-17|    14|
|대전|      53|     936|2022-01-21|    15|
|대전|      53|     928|2022-01-20|    16|
|대전|      53|     921|2022-01-19|    17|
|대전|      53|     936|2022-01-21|    18|
|대전|      53|     928|2022-01-20|    19|
+----+--------+--------+----------+------+
only sho

In [58]:
save_data(conf_svc, coFacPat, "CO_FACT_PATIENTS")

Py4JJavaError: An error occurred while calling o352.jdbc.
: java.sql.SQLException: Access denied for user 'bigSVC'@'localhost' (using password: YES)
	at com.mysql.cj.jdbc.exceptions.SQLError.createSQLException(SQLError.java:129)
	at com.mysql.cj.jdbc.exceptions.SQLExceptionsMapping.translateException(SQLExceptionsMapping.java:122)
	at com.mysql.cj.jdbc.ConnectionImpl.createNewIO(ConnectionImpl.java:828)
	at com.mysql.cj.jdbc.ConnectionImpl.<init>(ConnectionImpl.java:448)
	at com.mysql.cj.jdbc.ConnectionImpl.getInstance(ConnectionImpl.java:241)
	at com.mysql.cj.jdbc.NonRegisteringDriver.connect(NonRegisteringDriver.java:198)
	at org.apache.spark.sql.execution.datasources.jdbc.connection.BasicConnectionProvider.getConnection(BasicConnectionProvider.scala:49)
	at org.apache.spark.sql.execution.datasources.jdbc.connection.ConnectionProviderBase.create(ConnectionProvider.scala:102)
	at org.apache.spark.sql.jdbc.JdbcDialect.$anonfun$createConnectionFactory$1(JdbcDialects.scala:160)
	at org.a

### 4. 요일별 코로나 확진자 수를 구해보자
- 코로나 현황 data : STD_DAY 기준일 data -> 요일로 변경(dayofweek()) -> 요일별 그룹 생성후 신규 확진자수 합산

In [69]:
# 코로나 현황 data 필요
patients = find_data(conf_dm, "CORONA_PATIENTS")

# 기준일을 요일로 변환
patWeek = patients.withColumn("DAY_OF_WEEK", dayofweek(col("STD_DAY")))
patWeek.show(5)

+----+---------+-------+-----------+--------+----------+-----------+
| LOC|DEATH_CNT|DEF_CNT|LOC_OCC_CNT|QUR_RATE|   STD_DAY|DAY_OF_WEEK|
+----+---------+-------+-----------+--------+----------+-----------+
|강원|      110|  13838|         94|     899|2022-01-18|          3|
|서울|     2063| 243337|        684|    2559|2022-01-18|          3|
|대전|      187|  13262|        101|     913|2022-01-18|          3|
|전북|      121|  11094|        114|     621|2022-01-17|          2|
|광주|       55|  10049|        173|     697|2022-01-16|          1|
+----+---------+-------+-----------+--------+----------+-----------+
only showing top 5 rows



In [70]:
weekUp = patWeek.groupby(patWeek.DAY_OF_WEEK).agg(sum(col("LOC_OCC_CNT")).alias("PATIENTS"))
weekUp.show()

+-----------+--------+
|DAY_OF_WEEK|PATIENTS|
+-----------+--------+
|          1|  7626.0|
|          6| 12964.0|
|          3|  7526.0|
|          5| 12714.0|
|          4| 10862.0|
|          2|  7102.0|
+-----------+--------+



In [71]:
weekUp = weekUp.withColumn("DAY_OF_WEEK", when(weekUp.DAY_OF_WEEK == 1, "MON")
                                        .when(weekUp.DAY_OF_WEEK == 2, "TUE")
                                        .when(weekUp.DAY_OF_WEEK == 3, "WED")
                                        .when(weekUp.DAY_OF_WEEK == 4, "THE")
                                        .when(weekUp.DAY_OF_WEEK == 5, "FRI")
                                        .when(weekUp.DAY_OF_WEEK == 6, "SAT")
                                        .when(weekUp.DAY_OF_WEEK == 7, "SUN")
                 )
weekUp.show()

+-----------+--------+
|DAY_OF_WEEK|PATIENTS|
+-----------+--------+
|        MON|  7626.0|
|        SAT| 12964.0|
|        WED|  7526.0|
|        FRI| 12714.0|
|        THE| 10862.0|
|        TUE|  7102.0|
+-----------+--------+



In [72]:
# long -> wide 형으로 변경 후 저장
pdWeek = weekUp.pandas_api()
pdWeek

,DAY_OF_WEEK,PATIENTS
0,MON,7626.0
1,SAT,12964.0
2,WED,7526.0
3,FRI,12714.0
4,THE,10862.0
5,TUE,7102.0


In [74]:
pdWeekpvt = pdWeek.pivot_table(columns="DAY_OF_WEEK", values="PATIENTS")
pdWeek = pdWeekpvt.reset_index()

In [75]:
weekUp = pdWeek.to_spark()
weekUp.show()

/root/spark/python/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `to_spark`, the existing index is lost when converting to Spark DataFrame.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


+--------+-------+------+-------+-------+------+------+
|   index|    FRI|   MON|    SAT|    THE|   TUE|   WED|
+--------+-------+------+-------+-------+------+------+
|PATIENTS|12714.0|7626.0|12964.0|10862.0|7102.0|7526.0|
+--------+-------+------+-------+-------+------+------+



In [79]:
weekUp.withColumn("STD_DAY", lit(cal_std_day(365*3+4))).show()

+--------+-------+------+-------+-------+------+------+----------+
|   index|    FRI|   MON|    SAT|    THE|   TUE|   WED|   STD_DAY|
+--------+-------+------+-------+-------+------+------+----------+
|PATIENTS|12714.0|7626.0|12964.0|10862.0|7102.0|7526.0|2022-01-18|
+--------+-------+------+-------+-------+------+------+----------+



In [80]:
save_data(conf_svc, weekUp, "CO_WEEKDAY")

Py4JJavaError: An error occurred while calling o672.jdbc.
: java.sql.SQLException: Access denied for user 'bigSVC'@'localhost' (using password: YES)
	at com.mysql.cj.jdbc.exceptions.SQLError.createSQLException(SQLError.java:129)
	at com.mysql.cj.jdbc.exceptions.SQLExceptionsMapping.translateException(SQLExceptionsMapping.java:122)
	at com.mysql.cj.jdbc.ConnectionImpl.createNewIO(ConnectionImpl.java:828)
	at com.mysql.cj.jdbc.ConnectionImpl.<init>(ConnectionImpl.java:448)
	at com.mysql.cj.jdbc.ConnectionImpl.getInstance(ConnectionImpl.java:241)
	at com.mysql.cj.jdbc.NonRegisteringDriver.connect(NonRegisteringDriver.java:198)
	at org.apache.spark.sql.execution.datasources.jdbc.connection.BasicConnectionProvider.getConnection(BasicConnectionProvider.scala:49)
	at org.apache.spark.sql.execution.datasources.jdbc.connection.ConnectionProviderBase.create(ConnectionProvider.scala:102)
	at org.apache.spark.sql.jdbc.JdbcDialect.$anonfun$createConnectionFactory$1(JdbcDialects.scala:160)
	at org.a